In [1]:
import math
from typing import Tuple
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)  #nlayers 个encoder_layers
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)    #二维矩阵+偏置项

        self.init_weights()

    def init_weights(self) -> None:   #->None仅说明函数返回None，->用以说明函数返回类型
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)     # 执行PositionalEncoding的forward函数
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)  #triu生产三角矩阵，diagonal指定对角线位置

In [20]:
 torch.triu(torch.ones(4,4) * float('-inf'),diagonal=1)

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [2]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
src = torch.rand(10, 32, 512)
out = transformer_encoder(src)


In [35]:
embedding = nn.Embedding(10, 3)  #10对应参数input中的最大值
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1,2,4,5,3],[4,3,2,9,5]])
output=embedding(input)
print(input)
print(output.shape)
print(output.transpose(1,2).shape)
print(input)


tensor([[1, 2, 4, 5, 3],
        [4, 3, 2, 9, 5]])
torch.Size([2, 5, 3])
torch.Size([2, 3, 5])
tensor([[1, 2, 4, 5, 3],
        [4, 3, 2, 9, 5]])


tensor([[1, 4, 3],
        [4, 2, 5]])

In [36]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)  #随机按指定概率对矩阵元素设置为0

        position = torch.arange(max_len).unsqueeze(1)  #unsqueeze(1)一维数组转为2维
        #torch.exp:e的次方，math.log:默认底数为e
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)  #register_buffer添加一个缓冲器，暂不执行

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [80]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')  #tokenizer()将句子按照英文单词的进行拆分
#迭代句子，构建单词和数字间的映射，后面需要用这种映射将英文句子转换成数字,specials指定特殊字符，特殊字符生成的数字最小
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])  
vocab.set_default_index(vocab['<unk>'])

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]    #把英语句子按vocab映射为数字，iterm为英文句子，
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))   #cat：连接相同的序列张量，不指定dim，则默认结果为1维

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

# batch_size = 20
# eval_batch_size = 10
# print(train_data.shape)
# train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
# print(train_data.shape)
# val_data = batchify(val_data, eval_batch_size)
# test_data = batchify(test_data, eval_batch_size)


In [90]:
data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in test_iter]
torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

tensor([   9,  632,    0,  ..., 7213,    0,    3])

In [72]:
#vocab = build_vocab_from_iterator(map(tokenizer, "i am student"), specials=['<unk>'])  
tokenizer("i am student")
t=[torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in test_iter] 
print(t)

In [79]:
train_data[1].shape

torch.Size([20])

In [5]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)       #reshape(-1)将多维矩阵转为一维矩阵
    return data, target

In [9]:
data=torch.Tensor([[1,3,4],[2,3,5]])
data.reshape(-1)

tensor([1., 3., 4., 2., 3., 5.])

In [14]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [10]:
len(vocab)

28782

In [15]:
model.train()

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=200, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=200, out_features=200, bias=True)
        (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        

2

In [8]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt    # // 求整数
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [16]:
for batch, i in enumerate(range(0, 100, bptt)):
    print(batch,i)

0 0
1 35
2 70


In [9]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 346.76 | loss  8.11 | ppl  3318.03
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 342.45 | loss  6.88 | ppl   970.73
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 339.58 | loss  6.45 | ppl   632.49
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 320.95 | loss  6.31 | ppl   549.33
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 310.71 | loss  6.20 | ppl   490.38
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 309.14 | loss  6.16 | ppl   472.90
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 309.53 | loss  6.12 | ppl   453.66
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 307.71 | loss  6.11 | ppl   448.47
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 308.68 | loss  6.02 | ppl   411.33
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 310.71 | loss  6.02 | ppl   410.22
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 312.48 | loss  5.90 | ppl   364.65
| epoch   